In [1]:
import pandas as pd
import numpy as np
import re
# nltk.download('punkt')

In [2]:
df = pd.read_csv("D:/Hindi to English LSTM/preprocessed_hindi_to_english_dataset_maxlen15.csv")

In [3]:
df.head()

,Unnamed: 0,hindi,english
0,0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,give your application an accessibility workout
1,1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,accerciser accessibility explorer
2,2,निचले पटल के लिए डिफोल्ट प्लगइन खाका,the default plugin layout for the bottom panel
3,3,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका,the default plugin layout for the top panel
4,4,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष...,a list of plugins that are disabled by default


In [4]:
df.tail()

,Unnamed: 0,hindi,english
990160,990160,आप गीता के अध्ययन की अपेक्षा फुटबाल खेल कर स्व...,you will be nearer to heaven through football ...
990161,990161,आपको अपनी शिक्षा तभी उपयोगी लगेगी जब आप स्वयं ...,you will find your education useful only if yo...
990162,990162,आप अपनी जीवनवृत्ति में अनेक पेशेवर उपलब्धियां ...,you will have many professional accomplishment...
990163,990163,आप भारत की एकता और विविधता का सार्थक प्रतीक बन...,you will symbolise the unity and diversity of ...
990164,990164,जुबिन मेहता के संगीत में सीमाओं के आरपार पहुंच...,zubin mehta’s music has the power to transcend...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990165 entries, 0 to 990164
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  990165 non-null  int64 
 1   hindi       990165 non-null  object
 2   english     990164 non-null  object
dtypes: int64(1), object(2)
memory usage: 22.7+ MB


In [6]:
df_1 = df.drop(columns=['Unnamed: 0'])
dataset = df_1.iloc[:25000]

In [7]:
type(dataset)

pandas.core.frame.DataFrame

In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   hindi    25000 non-null  object
 1   english  25000 non-null  object
dtypes: object(2)
memory usage: 390.8+ KB


In [9]:
dataset.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,accerciser accessibility explorer
2,निचले पटल के लिए डिफोल्ट प्लगइन खाका,the default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका,the default plugin layout for the top panel
4,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष...,a list of plugins that are disabled by default


In [10]:
df = list(zip(dataset['english'], dataset['hindi']))

In [11]:
# print(df)

In [12]:
input_eng_lines = []
target_hin_lines = []
input_tokens = set()
target_tokens = set()
for lines in df:
    input_line, target_line = lines[0], lines[1]
#     input_line = [x for x in input_line if not (isinstance(x, float) and x.is_integer())]
    input_eng_lines.append(input_line)
    target_line = " ".join(re.findall(r"[\w']+|[^\s\w]", target_line))
    target_line = ' ' + target_line + ' '
    target_hin_lines.append(target_line)
    for token in re.findall(r"[\w']+|[^\s\w]", input_line):
        if token not in input_tokens:
            input_tokens.add(token)
    for token in target_line.split(" "):
        if token not in target_tokens:
            target_tokens.add(token)
input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)
input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])
reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())
max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", str(input_line))) for input_line in input_eng_lines])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_line)) for target_line in target_hin_lines])
encoder_input_data = np.zeros(
    (len(input_eng_lines), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_eng_lines), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_eng_lines), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
for line, (input_line, target_line) in enumerate(zip(input_eng_lines, target_hin_lines)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_line)):
        encoder_input_data[line, timestep, input_features_dict[token]] = 1
    for timestep, token in enumerate(target_line.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1
        if timestep > 0:
            decoder_target_data[line, timestep-1, target_features_dict[token]] = 1

In [13]:
print(max_encoder_seq_length, num_encoder_tokens)

15 1808


In [14]:
print(target_hin_lines[100],input_eng_lines[100], num_decoder_tokens, num_encoder_tokens)

 म ि ट ा न े क ी प ु ष ् ट ि कर े ं  confirm clear 758 1808


In [15]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)

(25000, 15, 1808)
(25000, 57, 758)


In [16]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
dimensionality = 500
batch_size = 128
epochs = 10

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

training_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
#training_model.save('training_model.h5')

In [17]:
# training_model.save('training_model.h5')

In [18]:
# training_model.summary()

NameError: name 'training_model' is not defined

In [19]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']
# loss = history.history['loss']
# val_loss = history.history['val_loss']

# epochs = range(len(acc))

# plt.plot(epochs, acc, 'g', label='Training accuracy')
# plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
# plt.title('Training and validation accuracy')
# plt.legend(loc=0)
# plt.savefig('wordA.png')
# plt.figure()
# plt.show()

# plt.plot(epochs, loss, 'r', label='Training loss')
# plt.plot(epochs, val_loss, 'o', label='Validation loss')
# plt.title('Training and validation loss')
# plt.legend(loc=0)
# plt.savefig('wordL.png')
# plt.figure()
# plt.show()

NameError: name 'history' is not defined

In [21]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model, load_model

training_model = load_model("training_model.h5")
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

latent_dim = 500
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

def decode_response(test_inputs):
    states_value = encoder_model.predict(test_inputs)
    target_seq = np.zeros((1,1,num_decoder_tokens))
    target_seq[0,0,target_features_dict['']] = 1.
    decoded_sentence = ''
    stop_condition = False
    while not stop_condition:
        output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_features_dict[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if(sampled_token == '' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [hidden_state, cell_state]
    return decoded_sentence

NameError: name 'dimensionality' is not defined

In [ ]:
training_model.summary()

In [ ]:
class Translator:
    exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later", "stop", "halt")
    def start(self):
        user_response = input("Give in an English sentence. ;) \n")
        self.translate(user_response)
    def translate(self, reply):
        while not self.make_exit(reply):
            reply = input(self.generate_response(reply) + "\n")
    def string_to_matrix(self, user_input):
        tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
        user_input_matrix = np.zeros(
            (1, max_encoder_seq_length, num_encoder_tokens),
            dtype='float32')
        for timestep, token in enumerate(tokens):
            if token in input_features_dict:
                user_input_matrix[0, timestep, input_features_dict[token]] = 1.
        return user_input_matrix
    def generate_response(self, user_input):
        input_matrix = self.string_to_matrix(user_input)
        chatbot_response = decode_response(input_matrix)
        chatbot_response = chatbot_response.replace("", '')
        return chatbot_response
    def make_exit(self, reply):
        for exit_command in self.exit_commands:
            if exit_command in reply:
                print("Ok, Until we meet again!")
                return True
        return False
translator = Translator()

In [ ]:
translator.start()

In [ ]:
model = load_model("training_model.h5")
